In [33]:
from datasets import load_dataset
from transformers import pipeline,set_seed
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
set_seed(43)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [16]:
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")
print(dataset['train'])
sample=dataset['train'][1]
len(sample['article'])

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})


4051

In [20]:
sample=dataset['train'][0]
len(sample['article'])

2527

In [24]:
sample_text=dataset['train'][1]['article'][:1000]
summaries={}
len(sample_text)

1000

In [ ]:
def baseline(text):
  return '\n'.join(sent_tokenize(text)[:3])

summaries['baseline']=baseline(sample_text)


In [34]:
pipe=pipeline('text-generation',model='gpt2-medium')
gpt_query=sample_text+'\nTL;DR:\n'
pipe_out=pipe(gpt_query,max_length=512)
summaries['gpt2']='\n'.join(sent_tokenize(pipe_out[0]['generated_text']))

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [35]:
summaries['gpt2']

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."\nHere, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court.\nMost often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies."\nHe says the arrests often result from confrontations with police.\nMentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely t

**T5**

In [37]:
pipe=pipeline('summarization',model='t5-small')
pipe_out=pipe(sample_text)
summaries['t5']='\n'.join(sent_tokenize(pipe_out[0]['summary_text']))
summaries['t5']


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


"inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court .\nmost often, they face drug charges or charges of assaulting an officer .\nmentally ill people become more paranoid, delusional, and less likely to follow dir ."

In [38]:
pipe=pipeline('summarization',model='google/pegasus-cnn_dailymail')
pipe_out=pipe(sample_text)
summaries['pegasus']=pipe_out[0]['summary_text'].replace('.<n>','.\n')
summaries['pegasus']

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cpu


'Mentally ill inmates are housed on the "forgotten floor" of a Miami jail .\nJudge: Most often, they face drug charges or charges of assaulting an officer .\nHe says the arrests often result from confrontations with police .\nMentally ill people often won\'t do what they\'re told when police arrive on the scene .'

In [44]:
print(dataset['train'][1]['highlights'])


Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [49]:
for model_name in summaries:
  print()
  print(model_name.upper())
  print()
  print(summaries[model_name])


GPT2

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."
Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court.
Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies."
He says the arrests often result from confrontations with police.
Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to fol

In [50]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [59]:
from evaluate import load
rouge=load('rouge')
real_sum=dataset['train'][1]['highlights'][:1000]
pred_sum=summaries['pegasus']
result=rouge.compute(predictions=[pred_sum],references=[real_sum])
print(result)

{'rouge1': np.float64(0.3921568627450981), 'rouge2': np.float64(0.16), 'rougeL': np.float64(0.2745098039215686), 'rougeLsum': np.float64(0.3529411764705882)}


In [60]:
real_sum=dataset['train'][1]['highlights'][:1000]
pred_sum=summaries['gpt2']

result=rouge.compute(predictions=[pred_sum],references=[real_sum])
print(result)

{'rouge1': np.float64(0.16055045871559634), 'rouge2': np.float64(0.06451612903225806), 'rougeL': np.float64(0.0963302752293578), 'rougeLsum': np.float64(0.14678899082568808)}


In [61]:
real_sum=dataset['train'][1]['highlights'][:1000]
pred_sum=summaries['baseline']

result=rouge.compute(predictions=[pred_sum],references=[real_sum])
print(result)

{'rouge1': np.float64(0.365079365079365), 'rouge2': np.float64(0.14516129032258066), 'rougeL': np.float64(0.20634920634920634), 'rougeLsum': np.float64(0.2857142857142857)}
